In [ ]:
import pandas as pd
import numpy as np
import os
import geopandas as gp

In [ ]:
# -- year being used for evaluation
Year = 2014

# -- spatial key being used for calculation
SpatialKey = 'ZIPCODE'

# -- input path
InputPath = '../../shapeData/Dataframes/'

# -- output path
OutputPath = '../../shapeData/Tables/'

In [ ]:
# -- read the park quality file on a year basis 
try:
    ParkQualityYear
except:
    print "reading %d_Inspections.csv..." % (Year)
    yearDF = pd.read_csv(InputPath + '%d_Inspections.csv' % (Year), sep='\t')

In [ ]:
# -- Get rid of the duplicated index column
del yearDF['Unnamed: 0']

In [ ]:
# -- categories
categories = [u'Athletic Fields', u'Benches',
       u'Fences', u'Glass', u'Graffiti', u'Horticultural Areas', u'Ice',
       u'Lawns', u'Litter', u'Paved Surfaces', u'Play Equipment',
       u'Safety Surface', u'Sidewalks', u'Trails', u'Trees', u'Water Bodies',
       u'Weeds']

In [ ]:
# -- calculate inspection scores [ratio]
yearDF['Ratings Ratio'] = 1 - yearDF[categories].mean(axis=1)

In [ ]:
# -- seed final park stats DF
byParkID = yearDF[['Prop ID', 'PID_base', 'Category'] + [SpatialKey]].groupby(['Prop ID'], as_index = False)
parkStats = byParkID.first()

# -- average Park Score and Acres
byParkBase = yearDF.groupby(['Prop ID'], as_index = False)
parkAvgScore = byParkBase['Ratings Ratio','ACRES_x'].mean()
parkAvgScore.rename(columns={'Ratings Ratio':'Avg Ratio'}, inplace=True)
parkStats = pd.merge(parkStats, parkAvgScore, on='Prop ID', how='left')

# -- calculate weighted Score for park
parkStats['Weighted Score'] = parkStats['Avg Ratio'] * parkStats['ACRES_x']

In [ ]:
# -- seed district_category stats DF
byCategory = parkStats.groupby([SpatialKey] + ['Category'], as_index = False)
categoryStats = byCategory.first()[[SpatialKey] + ['Category']]

# -- sum Acres and Scores per category
catSums = byCategory['ACRES_x', 'Weighted Score'].sum()
catSums.rename(columns={'Weighted Score':'Score'}, inplace=True)
categoryStats = pd.merge(categoryStats, catSums, on=[SpatialKey] + ['Category'], how='left')

# -- count parks per category
catCounts = byCategory.size().reset_index().rename(columns={0:'Counts'})
categoryStats = pd.merge(categoryStats, catCounts, on=[SpatialKey] + ['Category'], how='left')

In [ ]:
# -- group by zipcode and category
bySpatialKey = categoryStats.groupby(SpatialKey, as_index = False)
SpatialKeySums = bySpatialKey.sum()[[SpatialKey] + ['Counts']]

categoryStats = pd.merge(categoryStats, SpatialKeySums, on=SpatialKey, how = 'left', suffixes = ['_Cat', '_%s' % SpatialKey])

In [ ]:
# -- calculate weighted score for zipcode
categoryStats['Weighted Score'] = categoryStats['Score'] / categoryStats['ACRES_x']
categoryStats['Normalized Score'] = categoryStats['Weighted Score'] * (categoryStats['Counts_Cat'] / categoryStats['Counts_%s' % SpatialKey])

In [ ]:
# -- sum Scores per Zipcode
bySpatialKey = categoryStats.groupby(SpatialKey)
SpatialStats = bySpatialKey \
                    .sum() \
                    .reset_index() \
                    [[SpatialKey] + ['Normalized Score']]

In [ ]:
# -- Write to csv
SpatialStats.to_csv(OutputPath + 'ParkQuality_%s_%s.csv' % (SpatialKey, Year), sep=',')